In [61]:
# Find the Recipe Cover and Insert 

def recipeCover(soup,recode):
    cover = soup.find("img","img-hover")['src']
    print(cover)
    cursor.execute("UPDATE receipe SET pic=%s WHERE id=%s", (cover,recode))
    db.commit()
    return cover

In [112]:
# Insert the Recipe to the database  (serves didnt declare)

def insertRecipe(eachpage,soup,idex):
    title = soup.find("h1").text
    introduce = ""
    time = soup.find("small").text
    print(title)
    sqlcheck = 'SELECT * FROM receipe WHERE sources = 0 and url = %s'
    cursor.execute(sqlcheck,(eachpage))
    resultUrl = cursor.fetchone()
    if(resultUrl != None):
        recid = resultUrl[0]
        try:
            sql = 'INSERT INTO rec_type(id_rec,id_type) VALUES(%s,%s)'
            cursor.execute(sql,(int(recid),int(10)))
            db.commit()
            print("食譜類別更新!")
        except:
            pass
        return 1
    else:
        sql = 'INSERT INTO receipe(id,name,sources,introduce,times,url) VALUES(%s,%s,%s,%s,%s,%s)'
        sqllink = 'INSERT INTO rec_type(id_rec,id_type) VALUES(%s,%s)'
        # 执行sql语句
        cursor.execute(sql,(int(idex),title,int(0),introduce,time,eachpage))
        cursor.execute(sqllink,(int(idex),int(10)))
        
        # 提交到数据库执行
        db.commit()
        recipeCover(soup,idex)
        print("Success insert to 食譜")
        return 0


In [113]:
def titlereload(page,soup,idex):
    title = soup.find("h1").text
    print(title)
    sqltitle = 'INSERT INTO receipe(name) VALUES (%s)'
    cursor.execute(sqltitle,(title))
    db.commit()
    print("名稱重新load")

In [114]:
# Insert the each step of the recipe to database

def insertStep(soup,idex):
    div = soup.findAll("div","row mg-btm10")
    index = 1
    findit = 0
    
    
    for imgtag in div:
        if((imgtag.find('img'))!=None): # if it has picture
            src = imgtag.find('img')['src']
            content = imgtag.find('img')['title']
            #print(index)
            #print(src)
            #print(content)
            sql2 = 'INSERT INTO steps(id_receipe,step,content,pic) VALUES(%s,%s,%s,%s)'
            cursor.execute(sql2,(int(idex),int(index),content,src))
            db.commit()
            findit = 1
            index = index + 1
    if(findit == 0):    # if doesn't
        for span in div:
            step = span.find('span',"step_by")
            if(step):
                stepContent = step.find_next('p').text
                #print(index,":",stepContent)
                index = index + 1
                sql2 = 'INSERT INTO steps(id_receipe,step,content) VALUES(%s,%s,%s)'
                cursor.execute(sql2,(int(idex),int(index),stepContent))
                db.commit()
                #print("Success insert to 步驟")
        

In [115]:
def hashtag(soup,idex,auto):
    hashtag = soup.findAll('a','btn btn-default btn-outline')
    for eachtag in hashtag:
        tagname = eachtag.text
        sqlcheck = 'SELECT * FROM cate_receipe WHERE name = %s'
        cursor.execute(sqlcheck,(tagname))
        resultTag = cursor.fetchone()
        
        sqltag = 'INSERT INTO rec_cate(id_rec,id_cate) VALUES(%s,%s)'
        
        
        if(resultTag != None):
            tagId = resultTag[0]
            sqlduplicate = 'SELECT * FROM rec_cate WHERE id_rec = %s and id_cate = %s '
            cursor.execute(sqlduplicate,(int(idex),int(tagId)))
            duplicateTag = cursor.fetchone()
            if(duplicateTag == None):
                cursor.execute(sqltag,(int(idex),tagId))
                db.commit()
            
        elif(resultTag == None):
            sqlInsertTag = 'INSERT INTO cate_receipe(id,name) VALUES(%s,%s)'
            cursor.execute(sqlInsertTag,(int(auto),tagname))
            db.commit()
            sqltag = 'INSERT INTO rec_cate(id_rec,id_cate) VALUES(%s,%s)'
            cursor.execute(sqltag,(int(idex),int(auto)))
            db.commit()
            auto = auto + 1;
    return auto
            

In [116]:
# Check If the Seanson or Food exists in the database or not , if not 
# 若不存在則無條件加入 食材 code  ( or add to both table 食材 code and 調味料 code)
# 並且直接加入 食材 table 之中

def checkSeasoning(soup,idex):
    divouter = soup.findAll("div","row mg-btm10-border")
    for divinner in divouter:
        season = divinner.find("div","col-sm-8").text
        
        amount = divinner.find("div","col-sm-4").text
        
        # check the season
        sqlcheck = 'SELECT * FROM spice WHERE name = %s'
        cursor.execute(sqlcheck,(season))
        resultSeason = cursor.fetchone()
        
        
        # check the food
        sqlcheck1 = 'SELECT * FROM ingredient WHERE name = %s'
        cursor.execute(sqlcheck1,(season))
        resultFood = cursor.fetchone()
        
        if(resultSeason!=None):   # if it is Season
            seasoncode = resultSeason[0]
            sqlInsert123 = 'INSERT INTO rec_spi(id_rec,id_spi,amount) VALUES(%s,%s,%s)'
            cursor.execute(sqlInsert123,(int(idex),seasoncode,amount))
            db.commit()   
        elif(resultFood != None): # if it is Food 
            foodcode = resultFood[0]
            sqlInsert = 'INSERT INTO rec_ingre(id_rec,id_ingre,amont) VALUES(%s,%s,%s)'
            cursor.execute(sqlInsert,(int(idex),foodcode,amount))
            db.commit()
        elif(resultSeason == None and resultFood == None): # both not =>　it is new one 
            print(season)
            print("It should be added to one table")
            sqlAddFood = 'INSERT INTO temp(id_rec,name,amount) VALUES(%s,%s,%s)'
            cursor.execute(sqlAddFood,(int(idex),season,amount))
            
            db.commit()
            

In [117]:
def tempadding(soup,idex):
    divouter = soup.findAll("div","row mg-btm10-border")
    for divinner in divouter:
        season = divinner.find("div","col-sm-8").text
        
        amount = divinner.find("div","col-sm-4").text
        
        # check the season
        sqlAddFood = 'INSERT INTO temp(id_rec,name,amount) VALUES(%s,%s,%s)'
        cursor.execute(sqlAddFood,(int(idex),season,amount))    
        db.commit()

In [118]:
# 前置動作
# 修改 URL 和 頁數
# 更改類別代號

#!/usr/bin/python3
import requests
import lxml
from bs4 import BeautifulSoup
# import regex 
import re    
pattern = re.compile(r'recipe-shares.php')
import pymysql

# 打开数据库连接
db =  pymysql.connect(host='localhost',
                       user='root',
                       passwd='root123456',
                       db='program',
                       charset='utf8')
 
# 使用cursor()方法获取操作游标 
cursor = db.cursor()

#中式料理0　https://www.fooding.com.tw/recipe-list.php?mainclass=100002&subclass=100054&sort=a&page=2
#台式料理1  https://www.fooding.com.tw/recipe-list.php?mainclass=100002&subclass=100074&sort=a&page=
#西式料理2 https://www.fooding.com.tw/recipe-list.php?mainclass=100002&subclass=100058&sort=a&page=1000
#日式料理3 https://www.fooding.com.tw/recipe-list.php?mainclass=100002&subclass=100055&sort=a&page=2
#韓式料理4  https://www.fooding.com.tw/recipe-list.php?mainclass=100002&subclass=100056&sort=a&page=   
#餅乾麵包5  https://www.fooding.com.tw/recipe-list.php?mainclass=100002&subclass=100070&sort=a&page=
#舔點冰品6  https://www.fooding.com.tw/recipe-list.php?mainclass=100002&subclass=100060&sort=a&page=
#飲料調酒7  https://www.fooding.com.tw/recipe-list.php?mainclass=100002&subclass=100067&sort=a&page=400
#素食料理8  https://www.fooding.com.tw/recipe-list.php?mainclass=100002&subclass=100059&sort=a&page=411
#東南亞料理9 https://www.fooding.com.tw/recipe-list.php?mainclass=100002&subclass=100057&sort=a&page=500
# 其他 10   https://www.fooding.com.tw/recipe-list.php?mainclass=100002&subclass=100061&sort=a&page=6

sorturl = 'https://www.fooding.com.tw/recipe-list.php?mainclass=100002&subclass=100061&sort=a&page='

idex = 44419  # recipe # 

for p in range(1,49):
    pageurl = sorturl + str(p)
    print(pageurl)
    sortres = requests.get(pageurl)
    soupsort = BeautifulSoup(sortres.text,'html.parser')

    div = soupsort.find('div','col-md-9')
    # print(div)
    row = soupsort.findAll('div','row')
    link = 1
    for r in row:
        a = r.find_previous('a')
        try:
            if(a != None):
                result = re.match(pattern,a['href'])
                
                # Prepare to crawl each link
                if(result):    
                    eachpage = "https://www.fooding.com.tw/" + a['href']
                    print(eachpage)
                    link = link + 1
                    res = requests.get(eachpage)
                    soup = BeautifulSoup(res.text,'html.parser')
                    test = insertRecipe(eachpage,soup,idex)
                    if(test == 0):
                        insertStep(soup,idex)
                        tempadding(soup,idex)
                    
                        idex = idex + 1
        except KeyError:
            pass # or some other fallback action
            
# Send the page to BeautifulSoup to parser
#url = 'https://www.fooding.com.tw/recipe-shares.php?cookid=111155'
#res = requests.get(url)
#soup =  BeautifulSoup(res.text, 'html.parser')
#insertRecipe(soup)
#insertStep(soup)
db.close()

https://www.fooding.com.tw/recipe-list.php?mainclass=100002&subclass=100061&sort=a&page=1
https://www.fooding.com.tw/recipe-shares.php?cookid=114729
【型農大聯盟-欣奇農場青春閨蜜禮盒(木瓜禮盒)】【型農大聯盟】萊姆伏特加feat.木瓜
https://dd01iywm59npv.cloudfront.net/images/cookbooks/114729/cb114729_1543377702_c.jpeg


D:\anaconda\lib\site-packages\pymysql\cursors.py:170: Warning: (1364, "Field 'rank' doesn't have a default value")
  result = self._query(query)


Success insert to 食譜
https://www.fooding.com.tw/recipe-shares.php?cookid=114728
【私房食譜】啤酒蝦
https://dd01iywm59npv.cloudfront.net/images/cookbooks/114728/cb114728_1543285416_c.jpg
Success insert to 食譜
https://www.fooding.com.tw/recipe-shares.php?cookid=114724
【型農大聯盟-欣奇農場青春閨蜜禮盒(木瓜禮盒)】甜滋滋木瓜[欣奇農場青春閨蜜禮盒]
https://dd01iywm59npv.cloudfront.net/images/cookbooks/114724/cb114724_1543242581_c.JPG
Success insert to 食譜
https://www.fooding.com.tw/recipe-shares.php?cookid=114699
【型農大聯盟-欣奇農場青春閨蜜禮盒(木瓜禮盒)】木瓜試吃【型農大聯盟】
https://dd01iywm59npv.cloudfront.net/images/cookbooks/114699/cb114699_1542793421_c.jpg
Success insert to 食譜
https://www.fooding.com.tw/recipe-shares.php?cookid=114707
【型農大聯盟-欣奇農場青春閨蜜禮盒(木瓜禮盒)】【型農大聯盟-欣奇農場青春閨蜜禮盒(木瓜禮盒)】
https://dd01iywm59npv.cloudfront.net/images/cookbooks/114707/cb114707_1542975685_c.jpg
Success insert to 食譜
https://www.fooding.com.tw/recipe-shares.php?cookid=114703
【Hagoromo喜金雞油漬鮪魚罐頭L】家有上學兒的快速早餐-【喜金雞油漬鮪魚罐】
https://dd01iywm59npv.cloudfront.net/images/cookbooks/114703/cb114703_154

D:\anaconda\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'amount' at row 1")
  result = self._query(query)


https://www.fooding.com.tw/recipe-shares.php?cookid=113475
【Hagoromo喜金雞油漬鮪魚罐頭L】起司鮪魚南瓜【喜金雞】
https://dd01iywm59npv.cloudfront.net/images/cookbooks/113475/cb113475_1511694849_c.jpg
Success insert to 食譜
https://www.fooding.com.tw/recipe-shares.php?cookid=113466
【私房食譜】青蔥冷凍保存法 ～附『實作影片』～
https://dd01iywm59npv.cloudfront.net/images/cookbooks/113466/cb113466_1511409393_c.JPG
Success insert to 食譜
https://www.fooding.com.tw/recipe-shares.php?cookid=113463
【Hagoromo喜金雞油漬鮪魚罐頭L】鮪魚涼拌小黃瓜【Hagoromo喜金雞油漬鮪魚罐頭
https://dd01iywm59npv.cloudfront.net/images/cookbooks/113463/cb113463_1511343039_c.jpg
Success insert to 食譜
https://www.fooding.com.tw/recipe-shares.php?cookid=113453
【私房食譜】鮮奶起司燒
https://dd01iywm59npv.cloudfront.net/images/cookbooks/113453/cb113453_1511166248_c.jpg
Success insert to 食譜
https://www.fooding.com.tw/recipe-shares.php?cookid=113296
【私房食譜】幸福薑至-薑味嫩雞腿+情豆初開-感恩豆漿鍋
https://dd01iywm59npv.cloudfront.net/images/cookbooks/113296/cb113296_1506502985_c.jpg
Success insert to 食譜
https://www.fooding.com

...

https://www.fooding.com.tw/recipe-shares.php?cookid=109540
【私房食譜】海陸薑黃鮮粥【暖心料理】
食譜類別更新!
https://www.fooding.com.tw/recipe-list.php?mainclass=100002&subclass=100061&sort=a&page=15
https://www.fooding.com.tw/recipe-shares.php?cookid=109486
【私房食譜】免煮暖薑茶
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=109480
【私房食譜】綠咖哩蛋饅頭堡
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=109479
【私房食譜】【暖心料理】紅豆糕(電鍋版)
https://dd01iywm59npv.cloudfront.net/images/cookbooks/109479/cb109479_1475739874_c.JPG
Success insert to 食譜
https://www.fooding.com.tw/recipe-shares.php?cookid=109440
【私房食譜】薑汁枸杞地瓜粥【暖心料理】
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=109409
【私房食譜】客家桔醬
https://dd01iywm59npv.cloudfront.net/images/cookbooks/109409/cb109409_1448333215_c.jpg
Success insert to 食譜
https://www.fooding.com.tw/recipe-shares.php?cookid=109374
【私房食譜】【暖心料理】酒香桂圓紫米糕(電子鍋料理)
https://dd01iywm59npv.cloudfront.net/images/cookbooks/109374/cb109374_1475997801_c.JPG
Success insert to 食譜
https://www.food

【私房食譜】蜜糖早餐【蜂聖誕】
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=107283
【私房食譜】♦泰泰風♦艀豆芽-沒有漂白劑、催芽劑的豆芽教有味道!
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=107245
【私房食譜】【蜂聖誕】蜂蜜迷迭香烤雞腿
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=107246
【私房食譜】【蜂聖誕】蜂蜜香茅烤雞腿
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=107238
【私房食譜】【蜂聖誕】蜂蜜檸檬冰磚♥甜度酸度自己調配
食譜類別更新!
https://www.fooding.com.tw/recipe-list.php?mainclass=100002&subclass=100061&sort=a&page=22
https://www.fooding.com.tw/recipe-shares.php?cookid=107227
【私房食譜】咖哩地瓜雞
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=107200
【私房食譜】【蜂聖誕】蜂蜜香蕉煎餅
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=107196
【私房食譜】【蜂聖誕】檸檬蜜
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=107137
【私房食譜】【蜂聖誕】蜜堂─蜂蜜早午餐【vici的懶人廚房】
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=107090
【私房食譜】♦泰泰風♦洛神花果醬
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=107073
【將軍區農會牛蒡蔘、牛蒡茶】養生下午茶
食譜類別更新!
htt

【益康美食館黃金泡菜】運動過後的清爽料理~芙蓉豆腐泡菜沙拉
食譜類別更新!
https://www.fooding.com.tw/recipe-list.php?mainclass=100002&subclass=100061&sort=a&page=30
https://www.fooding.com.tw/recipe-shares.php?cookid=105073
【私房食譜】紅鳳菜麵條
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=105065
【益康美食館黃金泡菜】泡菜小黃瓜
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=104989
【私房食譜】馬鈴薯煎餅
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=105047
【益康美食館黃金泡菜】泡菜沙拉
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=105046
【益康美食館黃金泡菜】一口泡菜牛肉
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=105031
【私房食譜】花枝魷魚漿
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=105002
【私房食譜】馬鈴薯麻糬
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=104974
【私房食譜】輕食沙拉吐司
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=104913
【私房食譜】馬鈴薯煎餅
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=104871
【私房食譜】鹽炒花生
食譜類別更新!
https://www.fooding.com.tw/recipe-list.php?mainclass=100002&subcla

【田力精緻安全蔬果玉女小番茄】超甜田力玉女小番茄
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=102148
【私房食譜】冰焦糖瑪奇朵(Caramel Macchiatto)
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=102177
【田力精緻安全蔬果玉女小番茄】口感一級棒的玉女小番茄
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=102121
【甲仙地區農會初雲梅精】酸甘梅精
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=102105
【私房食譜】焦糖瑪奇朵(Caramel Macchiatto)
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=102056
【甲仙地區農會初雲梅精】黑森林金蕉佐梅精
食譜類別更新!
https://www.fooding.com.tw/recipe-list.php?mainclass=100002&subclass=100061&sort=a&page=39
https://www.fooding.com.tw/recipe-shares.php?cookid=102053
【糖郊赤崁糖手工原糖分享包】鮮奶茶
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=102044
【私房食譜】泡麵簡單煮
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=100433
【高雄六龜區農會金煌芒果】芒果牛奶冰沙
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=102012
【甲仙地區農會初雲梅精】紫山藥梅餅
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=101995
【甲仙地區農會初雲梅精】初雲梅精

【李雪辣嬌辣椒油潑辣子】麻辣乾炒牛河
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=100281
【高雄六龜區農會金煌芒果】消暑金煌芒果冰沙
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=100246
【高雄六龜區農會金煌芒果】全麥芒果紅茶貝果
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=100245
【台南楠西農芒頂級愛文芒果】天然ㄟ芒果醬 !!
食譜類別更新!
https://www.fooding.com.tw/recipe-shares.php?cookid=100215
【高雄六龜區農會金煌芒果】芒果冰沙
https://dd01iywm59npv.cloudfront.net/images/cookbooks/100215/cb100215_1374341306_c.JPG
Success insert to 食譜
https://www.fooding.com.tw/recipe-shares.php?cookid=100214
【高雄六龜區農會金煌芒果】金煌芒果冰沙
食譜類別更新!
https://www.fooding.com.tw/recipe-list.php?mainclass=100002&subclass=100061&sort=a&page=47
https://www.fooding.com.tw/recipe-shares.php?cookid=100209
【高雄六龜區農會金煌芒果】芒果醬
https://dd01iywm59npv.cloudfront.net/images/cookbooks/100209/cb100209_1374301969_c.JPG
Success insert to 食譜
https://www.fooding.com.tw/recipe-shares.php?cookid=100207
【高雄六龜區農會金煌芒果】芒果燉奶
https://dd01iywm59npv.cloudfront.net/images/cookbooks/100207/cb100207_1374301332

In [62]:
import requests
from time import gmtime, strftime

import lxml
from bs4 import BeautifulSoup
#!/usr/bin/python3
 
import pymysql
 
# 打开数据库连接
db =  pymysql.connect(host='localhost',
                       user='root',
                       passwd='root123456',
                       db='recipe',
                       charset='utf8')
 
# 使用cursor()方法获取操作游标 
cursor = db.cursor()


url = 'https://www.fooding.com.tw/recipe-shares.php?cookid=111161'
res = requests.get(url)
soup =  BeautifulSoup(res.text, 'html.parser')

def 


divouter = soup.findAll("div","row mg-btm10-border")

for divinner in divouter:
    season = divinner.find("div","col-sm-8").text
    amount = divinner.find("div","col-sm-4").text
    
    sqlcheck = 'SELECT * FROM 調味料code WHERE 名稱 = %s'
    
    sqlcheck1 = 'SELECT * FROM 食材code WHERE 名稱 = %s'
    cursor.execute(sqlcheck,(season))
    resultSeason = cursor.fetchone()
       
    cursor.execute(sqlcheck1,(season))
    resultFood = cursor.fetchone()
    if(resultSeason!=None):
        print(season)
        print(amount)
        print(resultSeason[0])
        seasoncode = resultSeason[0]
        sqlInsert123 = 'INSERT INTO 調味(食譜號碼,調味料號碼,用量) VALUES(%s,%s,%s)'
        cursor.execute(sqlInsert123,(int("2"),seasoncode,amount))
        db.commit()   
    elif(resultFood != None):
        print(season)
        print(amount)
        print(resultFood[0])
        foodcode = resultFood[0]
        sqlInsert = 'INSERT INTO 食材(食譜號碼,食材號碼,用量,食材分類) VALUES(%s,%s,%s,%s)'
        cursor.execute(sqlInsert,(int("2"),foodcode,amount,int("1")))
        db.commit()

# 关闭数据库连接
db.close()
    

SyntaxError: invalid syntax (<ipython-input-62-039052cf7c30>, line 25)

In [22]:
# 前置動作
#!/usr/bin/python3
import requests
import lxml
from bs4 import BeautifulSoup
# import regex 
import re    
pattern = re.compile(r'recipe-shares.php')
import pymysql

# 打开数据库连接
db =  pymysql.connect(host='localhost',
                       user='root',
                       passwd='root123456',
                       db='program',
                       charset='utf8')
 
# 使用cursor()方法获取操作游标 
cursor = db.cursor()

eachpage = 'https://www.fooding.com.tw/recipe-shares.php?cookid=106364'

idex = 349  # recipe # 
auto = 328
print(eachpage)
res = requests.get(eachpage)
soup = BeautifulSoup(res.text,'html.parser')
hashtag(soup,idex,auto)

https://www.fooding.com.tw/recipe-shares.php?cookid=106364


328

In [1]:
# introduction part (記得要在這邊把introduction加進資料庫) 5105 start

# 前置動作
#!/usr/bin/python3
import requests
import lxml
from bs4 import BeautifulSoup
# import regex 
import re    
pattern = re.compile(r'recipe-shares.php')
import pymysql

# 打开数据库连接
db =  pymysql.connect(host='localhost',
                       user='root',
                       passwd='root123456',
                       db='program',
                       charset='utf8')
 
# 使用cursor()方法获取操作游标 
cursor = db.cursor()

urlsql = 'SELECT * FROM receipe WHERE id >= 0'


cursor.execute(urlsql)
result = cursor.fetchall()
for row in result:
    url = row[9]
    recipeid = row[0]
    print(url)
    print(recipeid)
    res = requests.get(url)
    soupintro = BeautifulSoup(res.text,'html.parser')
    # print(soupintro.prettify())
    divTitle = soupintro.findAll("div","title-v1")
    for title in divTitle: 
        if(title.find_next_sibling('p')):
            introduce = title.find_next_sibling('p').text       
            print(introduce)
            introsql = 'UPDATE receipe SET introduce = %s where id = %s'
            cursor.execute(introsql,(introduce,recipeid))
            db.commit()
    
db.close()

https://www.fooding.com.tw/recipe-shares.php?cookid=104996
8030
天氣漸漸變熱了，完全不需要冰淇淋機，步驟簡單，在家也可以製作濃郁綿密冰淇淋(≧▽≦).
https://www.fooding.com.tw/recipe-shares.php?cookid=104961
8031
鳳梨含有膳食纖維、有機酸、維生素與蛋白酶等營養元素，其名稱的由來，根據台灣府誌記載，「葉薄而闊，而葉緣有刺，果生於葉叢中，果皮似波羅蜜而色黃，味甘而微酸，先端具綠葉一簇，形似鳳尾，故名」。

這道「鳳梨戀黑木耳凍」創意健康甜品，只要有鳳梨與一顆黑木耳健康凍，就能自行在家輕鬆完成，帶有酸甜、爽脆口感的鳳梨，搭配彈Q、滑口滋味的黑木耳健康凍，輔以些許煉乳來增味，一口吃下，多層次的風味，爽口又營養，鮮麗的成品色澤，更讓人在未動口前，就食慾倍增！擺盤時也只要花一點小巧思，也能將這道甜品融入時下話題，幻化成「太陽花」！
https://www.fooding.com.tw/recipe-shares.php?cookid=104837
8032
親子DIY晶瑩剔透繽紛亮麗好吃會"ㄉㄨㄞ ㄉㄨㄞ"的鮮果果凍，您是不是也對果凍愛不釋手呢？一起來感受果凍的魅力吧！
https://www.fooding.com.tw/recipe-shares.php?cookid=104832
8033
最近雷神巧克力在台灣很火紅！
但多年前吃過一次的我卻只覺得太甜不合胃口～”～
對於很愛吃有米果內餡巧克力的我突然興起自己動手做的念頭！
果然還是選用自己愛的食材做的對味！！
而且做法超簡單喔：）
https://www.fooding.com.tw/recipe-shares.php?cookid=104653
8034
簡單手做布丁+搭配香甜的的焦糖醬=幸福的滋味 ^^//
https://www.fooding.com.tw/recipe-shares.php?cookid=104641
8035
一口吃草莓煉乳冰，才放入口中，就立即化開來！濃濃的煉乳味~讚哪！
https://www.fooding.com.tw/recipe-shares.php?cookid=104635
8036
這低熱量的奶酪,雖然少了鮮奶油,但是一樣保有牛奶的香味.
孩子愛吃

最近的氣溫，似乎猶如溜滑梯一樣的直線下滑，讓人就想喝點或吃點熱騰騰的美食，來溫暖一下早已被到低溫寒流冷到小發抖的身心，因此今天分享的「黑Q紅豆牛奶甜湯」這道甜品料理，適合在這種冷ㄐㄐ的日子享用，由於黑木耳QQ圓屬果凍商品，雖不能直接加熱，卻可添加至已溫熱的食材中。
https://www.fooding.com.tw/recipe-shares.php?cookid=103578
8073
只是一到冬天，就讓人嘴饞想吃湯圓，但由於湯圓本身高熱量問題，常令愛吃的民眾望之卻步，這時不妨以黑木耳養生露為湯底，除協助減少熱量與糖分的攝取之外，又因黑木耳富含多種營養素，讓人吃的健康又養生！
https://www.fooding.com.tw/recipe-shares.php?cookid=103572
8074
近日受到大陸冷氣團的影響，全台各地氣溫明顯下降，平均氣溫約在13~16度之間，加上這波大路冷氣團挾帶豐沛的水汽，中部以北地區除降雨仍然明顯之外，其他地區也將出現短暫陣雨。

由於一直處在如此濕冷環境，總令人嘴饞地想吃些熱騰騰的美食，因此便跑去位於向上北路上的「統元豆花」，買了一碗熱的薑汁豆花回來解饞，但單吃豆花的感覺有點過於乏味，便再添加些許黑糖口味的黑木耳QQ圓，來增加其口感及滋味，而果不其然地，這樣神來一筆的玩味吃法，真的好吃太多了！！！
https://www.fooding.com.tw/recipe-shares.php?cookid=103531
8075
最近在怎樣的神經大條，也可以明顯感受到冬天寒冷的氣息，一到下午時刻，就想喀點熱食來暖和一下，於是就跑去中美街上「二姑姑的店」，買一杯椰香紅豆紫米粥回來解解饞。只是愛嚐鮮的我們，怎麼能放過玩味的機會，心想加一點熱鮮奶，應該會更好吃，再加一點溫熱的黑木耳養生露，感覺上又更營養一些，沒想到亂調味的結果，竟然出現三層漸層效果...
https://www.fooding.com.tw/recipe-shares.php?cookid=103499
8076
一提到可樂果，相信大家一定不陌生，
畢竟這可是看電影或嘴饞時的零食良伴，
然而這款早已為人所樂道的休閒食品，
最近不但將可樂果的外型變得超迷你，甚至還將口味變成甜的。

喜愛嚐鮮的我們，當然也不會錯過品嚐mini可樂果的箇中滋味，
但因為mini可樂果的

來自日本靜岡，Hagoromo Foods株式會社是日本是家喻戶曉的金牌食品品牌。2018年喜迎「喜金雞」成立60週年，Hagoromo Foods帶來日本人氣NO.1的「喜金雞鮪魚罐」「Hagoromo金黃甜玉米罐」以及便利美味的「Hagoromo義大利麵條」，還有口味眾多廣受歡迎的「Hagoromo義大利麵醬」。豐富台灣家庭的餐桌，料理只要簡單做就有最出色的美味。
https://www.fooding.com.tw/recipe-shares.php?cookid=110617
8121
最近身邊很多朋友 都在跟卡路里作戰
常常吃個飯 都要跟熱量斤斤計較
有時肚子餓、卻又不敢吃澱粉...
這種生活真的好辛苦呀

樂樂打算用麥片 取代米飯等澱粉類
既有飽足感又沒有太大的負擔
來看看今天的低卡無負擔晚餐吧^^

更多簡單好料理，歡迎按讚追蹤哦 https://www.facebook.com/leleskitchen
https://www.fooding.com.tw/recipe-shares.php?cookid=110570
8122
今天是端午節，大家都吃粽子了嗎？
沒有回娘家，也沒有回婆家
就我跟兩個孩子留守家中閉關
於是，為了給孩子有過節的氣氛
生平第一次，自己動手備料做了應景的粽子

用西谷米取代糯米的水晶粽
口感較為 Q 彈不傷胃。
我還加進了《蒟蒻晶米》
讓粽子吃得較無負擔且更有飽足感

所以，今天我們還是有吃到粽子
而且是難得一見的《銀粽》喔！！！

PS：哥哥還叫我要去為這粽子申請專利，是有這樣好吃嗎~ (掩嘴偷笑)

PS：這種水晶粽也很適合牙口不好的老年人取代糯米所帶來的不適感
不過，糖尿病患者不建議食用西谷米製品喔！！
https://www.fooding.com.tw/recipe-shares.php?cookid=110533
8123
之前分享了「海苔肉鬆免捏飯糰」，
這次試做其他口味~「醬燒魚片免捏飯糰」 :D
食譜分享：http://asd77723.pixnet.net/blog/post/43834354

（4人份，1人份約340大卡）
https://www.fooding.com.tw/recipe-shares.php?cookid=110467
8124
蝶豆藍花，在6年前， 
經我不斷的在網路放送它在食品

自己在家也可做出超美味的冰淇淋，這個食譜不用冰淇淋機器，只需三種材料，方便又簡單。這次我做Cookies & Cream的口味，就是加入OREO餅乾碎，自己做什麼口味也可，沒有防腐劑及色素，新鮮美味！快點自己做，消消暑囉！
請也到我的Facebook 專頁按讚唷^O^ 
http://www.facebook.com/MrsPsKitchen1
https://www.fooding.com.tw/recipe-shares.php?cookid=108740
8161
有次偶然中看到用蘑菇做mini漢堡的影片，就覺得好可愛！準備的東西不多就試著做看看沒想到，沒想到真的很可愛，而且是很適合做派對中finger food的料理，一口一個剛剛好，還很美味呢!!
http://on.fb.me/1MsnXJf
https://www.fooding.com.tw/recipe-shares.php?cookid=108767
8162
泰式涼拌菜是把魚露椰糖檸檬汁和著蔬菜海鮮拌一拌吃 
另與涼拌菜齊名的的是蔬菜沾著醬吃的『卡必醬沙拉』 
非常普遍的國民食物就是 『一口糯米一口魚 蔬菜沾著卡必醬吃』 
『卡必醬』的做法是『蔥頭+蒜頭+椰糖+魚露+檸檬+辣椒+蝦膏』 
而『泰泰風的蝦醬』本就包含上述食材 
所以用泰泰風蝦醬製做卡必醬就非常的方便 
這樣一餐下來飽足營養都有了 重點是低熱量 
難怪泰國胖子不多
https://www.fooding.com.tw/recipe-shares.php?cookid=108766
8163
早餐：蕎麥粥
午餐：炒蕎麥麵
晚餐：蕎麥飯

昨天吃了一天的蕎麥
應弟弟的要求
今天早上媽媽還是煮了好吃的蕎麥粥。
https://www.fooding.com.tw/recipe-shares.php?cookid=108749
8164
有時會想做一些，簡單但很好吃，一口一個的小點心，蘑菇真的是做這類型鹹小點心的最佳主角，去好梗，撒上番茄、起司、培根，加上橄欖油、胡椒，送進烤箱中180度15分鐘，完成!!
https://www.fooding.com.tw/recipe-shares.php?cookid=108723
8165
這道「蒜味番茄冷麵」(不流汗，簡單4步驟完成)把食材放入烤箱烤一下，全部打碎放冷，與義大利麵拌一拌就完成了簡單

最近在網路上陸續看到許多關於燕麥粥的食譜，自己也忍不住手癢，嚐試將黑木耳養生露與燕麥片結合，進而完成這道「黑木耳燕麥核桃粥」創意料理，燕麥片是大家熟悉的養生食材，加入鮮奶與黑木耳養生露小火煮成粥，不但能補充鈣質，只要再酌量添加桂圓與核桃來增添口感，花不到五分鐘的時間，一份營養又好呷的早餐，即能輕鬆端上桌享用！
https://www.fooding.com.tw/recipe-shares.php?cookid=107644
8207
以前小編冬天去泡溫泉都會喝到黑糖薑汁地瓜湯，一喝下去，薑絲讓身體馬上暖呼呼的。小編還加入對補血很好的紅棗 、枸杞，搭配黑糖喝起來甜而不膩，另外地瓜還能幫助腸胃順暢，吃甜湯同時吃到健康，這道料理只要電鍋就能完成，既養生又美味。
https://www.fooding.com.tw/recipe-shares.php?cookid=107643
8208
天氣冷颼颼，路上的薑母鴨每天都大排長龍，薑含有豐富的薑辣素，不論是幫助消化或促進新陳代謝非常有用，可以馬上讓身體暖呼呼。
https://www.fooding.com.tw/recipe-shares.php?cookid=107572
8209
簡單又好看的柳丁，讓吃柳丁多了小小的樂趣！

詳細步驟http://dale1128.pixnet.net/blog/post/444237944323408 
粉絲團：https://www.facebook.com/dale168
https://www.fooding.com.tw/recipe-shares.php?cookid=107502
8210
[海鮮肉類]
趁著連假好天氣小朋友在自家田裡烤地瓜.窯烤味道與烤箱風味不同.雞腿經由泥土的熱悶熟.袋口打開香氣撲鼻而來.只能說.....讚啦!
https://www.fooding.com.tw/recipe-shares.php?cookid=107490
8211
年節快到了~對要準備年菜的婆婆媽媽們而言,
可以預先準備好一兩道冷盤~~
除夕夜裡再也不用手忙腳亂做年夜飯料理啦!!
https://www.fooding.com.tw/recipe-shares.php?cookid=107487
8212
簡單營養食譜. 

創新、顛覆傳統三角飯糰: 運用調合自製美乃茲、味噌的柔

...

記得哥哥以前玩了好幾十罐的培樂多黏土..
現在弟弟也到了愛玩黏土的年紀了~~

差別在阿木我現在很會持家..
所以~~ 自己動手來做(培樂多)黏土吧!!
https://www.fooding.com.tw/recipe-shares.php?cookid=105206
8313
【馬鈴薯炊飯】

馬鈴薯燜飯  雖然簡單，卻又不平凡

比起白米來說多了一份口感，添加了份層次

同時也增添了些許的香氣


by 熊貓眼女孩 
https://www.facebook.com/aquapeng
https://www.fooding.com.tw/recipe-shares.php?cookid=105191
8314
這道菜的準備工作
1.小黃瓜先用削皮的器具，把小黃瓜削出薄片，先泡水備用
2.五鼓米煮好，備用（這裡可以用點壽司醋調味）
3.準備好要鋪在表面的餡料，可以是肉鬆，鮪魚之類的
準備好只要組裝就完成囉
https://www.fooding.com.tw/recipe-shares.php?cookid=105161
8315
這次小編受「億興米廠-米罐子短梗米」之邀，收到包裝精美的米!精選的台南11號及台中192號等優質米，聽說這種米做成壽司或飯糰再適合不過呢!!最適合中午帶便當吃!!今天就來做兒童飯糰餐吧!!
https://www.fooding.com.tw/recipe-shares.php?cookid=105156
8316
滿滿的料..
營養百分~
https://www.fooding.com.tw/recipe-shares.php?cookid=105154
8317
又酸又甜的芒果青 就像初戀的滋味~
讓人揪心呀!!
https://www.fooding.com.tw/recipe-shares.php?cookid=105125
8318
壽喜燒是一種不喝湯的火鍋

可以多準備一些蘿蔔、海帶、年糕等等喜歡的食材一起煮
https://www.fooding.com.tw/recipe-shares.php?cookid=105091
8319
喜愛海鮮的朋友不可錯過的料理
只要食材新鮮隨意煮都很可口的~
https://www.fooding.com.tw/recipe-shares.php?cookid=105060
8320


這是情人節甜點,在草莓薄荷沙拉食譜圖中有出現,不知你們有注意到? 當然也是為了慶祝我們的情人節, 少不了巧克力的誘惑, 還做了各種形狀的.口味我選擇櫻桃和杏仁果, 多做些不僅可以存放在保鮮盒,朋友來時可招待的下午茶點心.
https://www.fooding.com.tw/recipe-shares.php?cookid=104007
8377
2021社會企業-老梅果醬是我吃過最好吃的果醬了，實在是太好吃了啦！ 因為「2021社會企業-老梅果醬」用的梅子是自然農法栽植，完全不使用農要跟化學肥料，經由日本和歌山獨家技術的十年老梅精緻手工製成，完全不添加化學添加物以及果膠。 吃起來有古早味的鹹酸甘，微甜的滋味甜而不膩，微酸的滋味是那股迷人圓潤陳年老梅的香氣，用來做Party食物很適合喔！
https://www.fooding.com.tw/recipe-shares.php?cookid=100128
8378
蓋飯又稱蓋澆飯、燴飯與碟頭飯，是一種在日常生活中常見的米飯料理，蓋飯的主要特點在於飯與菜的結合，既有主食米飯，又有美味菜餚，不像其他餐點需要使用到較多的餐具碗盤，只要將飯與菜盛於一個碗內，馬上就可以放在桌上食用，也可以用手端著吃。
https://www.fooding.com.tw/recipe-shares.php?cookid=103904
8379
西洋情人節前菜系列.
今日前菜, 使用北非有名的couscous, 加入天然的蔬果, 只要10分鐘,做簡單健康的沙拉, 清爽好吃. 根據最近研究發現多吃番茄…等抗氧化蔬果,可幫助甩肉,讓體重0負擔. 這食譜中可加入個人喜歡的蔬菜,雞肉…
https://www.fooding.com.tw/recipe-shares.php?cookid=103305
8380
阿金姐姐妹紅潤釀是以傳統的紅麴加上糯米為主製成的醬料，擁有艷麗色澤、香氣與甘甜美味的紅麴味甘性溫、健脾益氣，而現代醫學科技發達，經實驗證實，釀造紅麴所產生的紅麴菌會產生種對人體有益的重要物質。

利用紅麴養身，紅潤釀略顯甜味的特殊香氣，不但能增添食物味道，使色澤鮮艷，更令人食欲大增。
https://www.fooding.com.tw/recipe-shares.php?cookid=103367
8381
簡單營養食譜. 共做5杯.採用微波

周一的中午懶洋洋，簡單弄個燉飯，營養美味 
妳也來弄一盤吧^_^

woman煮食生活家粉絲專頁 
https://www.facebook.com/womancooking 

喜歡的話請幫我按讚喔^_^
https://www.fooding.com.tw/recipe-shares.php?cookid=102558
8434
冬天總是喜歡吃烤馬鈴薯, 不論焗烤, 加茄汁, 絞肉或多種蔬菜加點調味料的食譜都喜歡.這食譜用橄欖油取代奶油(比較健康), 加入百里香拌一拌,放入烤箱烤熟即可,只要三步驟,簡單好吃的料理. 健康食譜
https://www.fooding.com.tw/recipe-shares.php?cookid=102530
8435
過了用餐時間,肚子餓時來一碗古早味的豬油醬油拌飯,勾起了童年的記憶,難忘如此的美味啊~飯後再來杯陽台的薄荷葉沖泡杯熱綠茶.真的是享受不在花錢多啊~~
https://www.fooding.com.tw/recipe-shares.php?cookid=102553
8436
簡單有營養的食譜. 先在鍋中炒香食材,加了薏仁和米飯ㄧ起在電鍋中煮熟. 健康又好吃
https://www.fooding.com.tw/recipe-shares.php?cookid=102482
8437
產季對的大刈菜做長年菜真的無敵美味!

長年菜熅一大鍋長年菜，取其長壽吉利之意。

我娘家沒有這習俗，但在婆家卻是每逢過年必吃的一道年菜^^
https://www.fooding.com.tw/recipe-shares.php?cookid=102509
8438
單吃或配吐司或饅頭都很棒唷~
https://www.fooding.com.tw/recipe-shares.php?cookid=102493
8439
生活壓力及三餐不正常或蔬果攝取的少..
吃進去的東西很容易囤積在身體..
昨天進食的東西如果沒有在隔天消化代謝出去的話..
肚子一整天都脹脹的不舒服..
雖然天天一起床會喝300cc左右的水..
但有時還是感覺不是那麼順暢..
偶而還是要補充發酵的東西..
例如：優酪乳、優格、乳酸菌、泡菜、鳳梨..
增加腸胃裡的好菌..
順暢了人就輕鬆又舒服~
https://www.fooding.com.tw/recipe-s

美濃好米美濃香鑽-高雄147

收到試吃品之後~一打開靠近聞~
就有聞到淡淡的芋頭香,
米飯煮好之後香味更是明顯
QQ彈牙~粒粒飽滿...
怎麼吃都好吃...

這次打算來做做簡單的貓飯
相信看過深夜食堂的人都會對這一道料理有印象
簡單而且溫暖~^^
https://www.fooding.com.tw/recipe-shares.php?cookid=101281
8502
使用美福食集紐西蘭濕式熟成牛排，搭配自己喜歡的食材來燉牛肉，烹飪時光聞牛肉香氣就好滿足喔!!
https://www.fooding.com.tw/recipe-shares.php?cookid=101319
8503
為了家人選擇安全食材..
吃了健康安心有保障..
支持優質農產品唷~

很開心能品嚐這麼棒的元氣產品..
讓我們一起輕鬆吃出健康有活力吧~

完整料理請到我的網誌看看：
http://beauty.pchome.com.tw/myroom/amcolove/article/1325602712
https://www.fooding.com.tw/recipe-shares.php?cookid=101299
8504
颱風又要來了嗎??
菜價漲又不停嗎??
便宜省錢的家常菜~~
健康美味無負擔的萬味香醬園黑豆白曝蔭油
同時也吃得到海味喲~~
https://www.fooding.com.tw/recipe-shares.php?cookid=101295
8505
米粒晶瑩透亮，烹煮飯香滿溢，吃一口滿滿芋頭香味十足!!
https://www.fooding.com.tw/recipe-shares.php?cookid=101234
8506
以柴魚醬油及菇類香氣做為基底的日式炊飯，可以因應季節變化，加入當令食材，變化多端而且健康養生。
https://www.fooding.com.tw/recipe-shares.php?cookid=101232
8507
紅文旦的營養價值

根據學者對紅文旦研究顯示，總酚、胡蘿蔔素、維生素C、維生素E等含量，皆高於白文旦，即使加熱，抗熱性也較佳，可說是營養價值很高的水果。

吃文旦的時機

文旦屬於應景的水果，以傳統節氣來說，從白露只是籽比較多也大了些..到中秋這段期間才可以採收。剛摘下的文旦果肉硬，放在室溫中一段時間，等到果皮

蒸飯是指把飯和菜放在一起蒸的飯，是我們平時最常用的烹調手法，只要將所有食材通通丟到電鍋中，等待一小段時間，就可以熱騰騰吃到有飯、也有菜的料理，雖然簡樸，卻能飽餐一頓，比起煮一鍋白飯又要張羅一桌菜，還要來得輕鬆許多...
https://www.fooding.com.tw/recipe-shares.php?cookid=100169
8567
別以為白米只能煮白米飯&粥，其實用一點點日式的做法加上旬味的食材依舊可以讓白米變身非常精彩的炊飯，現在正綠竹筍的季節搭配了毛豆等夏日的時蔬煮一鍋香噴噴的「雞肉竹筍炊飯」，這一碗可以吃到；香甜多汁的筍、Q彈的飯、豐富的時蔬，令人好滿足。
https://www.fooding.com.tw/recipe-shares.php?cookid=100151
8568
銀川米有機白米-強調"這將是您有史以來吃過最鮮的米"!!
這真的一點都沒錯~
小家庭僅用簡單的電鍋料理,來去呈現與有機米的搭配~
香!Q!入口即化~~~~好幸福!
https://www.fooding.com.tw/recipe-shares.php?cookid=100130
8569
這道料理簡單易學，食材也容易準備，偶爾提不起勁煮飯或沒食慾時是不錯的選擇，烹飪菜鳥也可輕鬆完成料理唷  (^0^) /

銀川有機白米 的生產環境乾淨，施用有機天然肥料；並且絕不使用農藥及化學肥料下成長茁壯，米中甘味的游離氨基酸很高，因而煮出的米飯光亮甘甜，入口時在喉頭與舌間所產生的風味豐富，因而讓人愛不釋口，冷飯一樣可口香軟好吃。
https://www.fooding.com.tw/recipe-shares.php?cookid=100122
8570
原本腦袋瓜想了好多類型的"芒果創意料理"，結果一一被林熙先生打槍。林熙先生說，這麼棒的芒果，一定要忠於原味，把芒果本身的香甜多汁的優點展現出來，不然太對不起品質這麼高水準的芒果了。

要展現芒果的香甜多汁，而且要忠於原味...
那就做林熙先生家鄉風味的GELATO吧:)

>取名 [澳客香芒GELATO]
https://www.fooding.com.tw/recipe-shares.php?cookid=100109
8571
芒果+優格
真的是健康又清爽的夏日好滋味!
優格選原味的就可以
因為芒果真的好甜好甜好甜~~~


In [7]:
# introduction part (記得要在這邊把introduction加進資料庫) 5105 start

# 前置動作
#!/usr/bin/python3
import requests
import lxml
from bs4 import BeautifulSoup
# import regex 
import re    
pattern = re.compile(r'recipe-shares.php')
import pymysql

# 打开数据库连接
db =  pymysql.connect(host='localhost',
                       user='root',
                       passwd='root123456',
                       db='program',
                       charset='utf8')
 
# 使用cursor()方法获取操作游标 
cursor = db.cursor()

urlsql = 'SELECT * FROM receipe WHERE id >= 3193'

    
    
cursor.execute(urlsql)
result = cursor.fetchall()
for row in result:
    url = row[9]
    recipeid = row[0]
    print(url)
    print(recipeid)
    res = requests.get(url)
    soupintro = BeautifulSoup(res.text,'html.parser')
    # print(soupintro.prettify())
    
    title = soupintro.find("h1").text
    print(title)
    introsql = 'UPDATE receipe SET name = %s where id = %s'
    cursor.execute(introsql,(title,recipeid))
    db.commit()
    print("名稱重新load")
    
db.close() 
    


https://www.fooding.com.tw/recipe-shares.php?cookid=106898
3193
【私房食譜】[菜籃食材] 暖呼呼ㄉ壽喜燒
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=106887
3194
【私房食譜】奶香紅豆泥球【菜籃食材】
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=106882
3195
【長滿園大甲香芋】香芋排骨粥
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=106871
3196
【私房食譜】什錦咖哩百匯
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=106851
3197
【私房食譜】快速即食南瓜燉飯(2人份)❤懶人食譜❤
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=106824
3198
【私房食譜】[菜籃食材]-紅白蘿蔔燉牛肉
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=106823
3199
【私房食譜】【LEHO台灣良品】干貝烏魚子XO醬蒸白菜
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=106821
3200
【私房食譜】【LEHO台灣良品】辣辣的蒟蒻
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=106820
3201
【LEHO 台灣良品LEHO《嚐。原釀》蘭級清油】【LEHO台灣良品】日式筑前煮
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=106732
3202
【私房食譜】LEHO <嚐。原釀> 蘭級清油蔭油│古早味燉雞
名稱重新load
https://www.fooding.com.tw/recipe

【私房食譜】豐年農場 - 黑棗燉猴頭菇瘦肉清湯
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=104683
3280
【私房食譜】薑黃蔬菜燉飯
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=104652
3281
【私房食譜】紅酒燉牛肉
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=104612
3282
【私房食譜】義式番茄肉醬
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=104596
3283
【豐年農場新鮮猴頭菇】菇菇雞湯
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=104594
3284
【私房食譜】菇菇燉雞
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=104556
3285
【私房食譜】竹笙香菇雞湯~巧活黑鑽雞
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=104537
3286
【私房食譜】【創意米飯】【禾飽滿滿有機米3月預購現碾】甜米球
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=104536
3287
【私房食譜】【創意米飯】【禾飽滿滿有機米3月預購現碾】黃彩米地瓜簽飯
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=104480
3288
【私房食譜】番茄紅醬燴飯
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=104478
3289
【巧活食品巧活黑鑽雞+清胸肉+三節翅 +原味雞高湯+黃金酥餅】黑木耳青木瓜雞湯粥
名稱重新load
https://www.fooding.com.tw/recipe

【私房食譜】紅燒牛肉湯
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=102935
3368
【私房食譜】紅蘿蔔燉牛肉
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=102934
3369
【私房食譜】【創意米飯】【禾飽滿滿有機米3月預購現碾】~五穀飯~美味營養無負擔
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=102872
3370
【吳秀乾農園梅乾菜】梅干白菜鴨肉鍋
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=102845
3371
【私房食譜】普羅旺斯燉菜 Ratatouille
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=102730
3372
【私房食譜】【創意米飯】【禾飽滿滿有機米3月預購現碾】麻油米糕
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=102789
3373
【晶饌美食家古味鵝油金蔥】鵝油金蔥雞腿飯
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=102767
3374
【晶饌美食家古味鵝油金蔥】鵝油金蔥白菜滷
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=101755
3375
【私房食譜】養生黑米粥
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=102640
3376
【私房食譜】蜜紅豆
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=102632
3377
【私房食譜】養生調理~桂圓四物雞湯
名稱重新load
https://www.fooding.com.tw/recipe-share

【田寮區農會果樹產銷班第十一班林慶揮蔭鳳梨】蔭鳳梨燉豆包
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=100475
3460
【田寮區農會果樹產銷班第十一班林慶揮蔭鳳梨】甘甜‧鳳梨苦瓜雞腿湯
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=100462
3461
【信功豬肉健康梅花豬肉】豬肉南瓜菇菇炊飯
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=100425
3462
【銀川米有機白米】おやこどん-搭配銀川米有機白米煮出來的白飯作成的親子丼
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=100369
3463
【銀川米有機白米】嬰兒副食品1:10倍粥
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=100328
3464
【丁媽媽ㄟ店鮮奶白蝦】燒酒蝦
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=100292
3465
【丁媽媽ㄟ店鮮奶白蝦】燒酒藥膳蝦
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=100264
3466
【銀川米有機白米】紅豆龍眼粥
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=100251
3467
【吳秀乾農園客家酸菜】辣味酸菜水煮雞
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=100238
3468
【民豐有機米有機珍珠香米】民豐有機米有機珍珠香米試吃
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=100235
3469
【民豐有機米有機珍珠香米】民豐有機米有機珍珠香米
名稱重新load
ht

【私房食譜】開胃爽口嫩薑蒸豆腐鮭魚【薑來會幸福x水果歐爸】
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=108022
3550
【私房食譜】鱸魚蛤蜊蒸豆腐【薑來會幸福x水果歐爸】
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=107964
3551
【私房食譜】純手工湯‧包‧燒賣
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=107910
3552
【私房食譜】焦糖雞蛋布丁[紫羽小廚甜品]
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=107887
3553
【私房食譜】和風藍鑽蝦
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=107756
3554
【私房食譜】蘿蔔糕
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=107710
3555
【私房食譜】瑪莉廚房：會爆漿的《小籠湯包》
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=107696
3556
【私房食譜】日式茶碗蒸
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=107641
3557
【私房食譜】絕對秒殺的《甜甜圈&貓爪軟糕》
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=107626
3558
【私房食譜】茄汁高麗菜盅
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=107558
3559
【私房食譜】白菜封肉【菜籃裡的海鮮肉類】
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=1075

...

【私房食譜】貢丸湯+貢丸湯麵
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=106841
8246
【私房食譜】Halloween - 邪惡傑克南瓜燈
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=106794
8247
【私房食譜】【好聚。沖繩】水雲涼拌鮭魚卵
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=106792
8248
【私房食譜】【好聚。沖繩】黑糖布丁
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=106784
8249
【【好聚。沖繩】水雲&沖繩垣乃花粉末黑糖】【好聚。沖繩】小熊黑糖紅豆吐司
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=106763
8250
【【好聚。沖繩】水雲&沖繩垣乃花粉末黑糖】【好聚。沖繩】水雲蔬菜粥
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=106522
8251
【【好聚。沖繩】水雲&沖繩垣乃花粉末黑糖】【好聚。沖繩】水雲高麗菜豬肉水餃
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=106716
8252
【【好聚。沖繩】水雲&沖繩垣乃花粉末黑糖】【好聚。沖繩】沖繩黑糖白飯麻糬
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=106693
8253
【【好聚。沖繩】水雲&沖繩垣乃花粉末黑糖】【好聚。沖繩】水雲醋飯糰
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=106689
8254
【私房食譜】家常版(韓式泡菜炒年糕)
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=106

【芳榮米廠禾家益全香米】壽司
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=104849
8333
【味一食品鮪魚鬆】魚鬆蔬菜粥
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=104841
8334
【私房食譜】♦泰泰風♦餐桌上的綠紅盤飾這樣做
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=104830
8335
【味一食品鮪魚鬆】美味鮪魚鬆花壽司
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=104816
8336
【私房食譜】燜南瓜(台灣好農-有機南瓜)
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=104740
8337
【味一食品鮪魚鬆】鮪魚鬆蓋飯
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=104715
8338
【田力精緻安全蔬果玉女小番茄】田力精緻安全蔬果玉女小番茄
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=104656
8339
【祥記梅子茶仔油】茶油乾拌麵
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=104634
8340
【味一食品鮪魚鬆】糙米鮪魚飯糰
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=104623
8341
【味一食品鮪魚鬆】鮪魚鬆拌豆腐
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=104618
8342
【私房食譜】酒香白斬雞
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=104613
8343
【私

【村家味花粉絹絲方便包】村家味花粉絹絲麻醬麵
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=102840
8420
【私房食譜】肉鬆夾心餅乾。
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=102819
8421
【村家味花粉絹絲方便包】餛飩麻醬麵
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=102262
8422
【田力精緻安全蔬果玉女小番茄】冰糖玉女小番茄
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=102813
8423
【田力精緻安全蔬果玉女小番茄】田力玉女小番茄
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=102772
8424
【私房食譜】花椒油
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=102748
8425
【田力精緻安全蔬果玉女小番茄】田力玉女小番茄
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=101871
8426
【私房食譜】日式蒸蛋
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=102722
8427
【村家味花粉絹絲方便包】村家味花粉絹絲, 簡單方便麵
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=102717
8428
【田力精緻安全蔬果玉女小番茄】梅汁小番茄
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=101462
8429
【私房食譜】咖哩飯
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=102375

【台灣好農元氣箱輕鬆嚐鮮組】有機紅文旦柚
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=101219
8508
【祥益食品即食沖泡式米粉-油蔥口味(葷)】祥益食品即食沖泡式米粉
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=101211
8509
【美濃好米美濃香鑽---高雄147】軟軟QQ好米
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=101176
8510
【台灣好農元氣箱輕鬆嚐鮮組】人妻的廚房-飯包蛋
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=101138
8511
【台灣好農元氣箱輕鬆嚐鮮組】紅文旦&麻豆文旦
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=101136
8512
【台灣好農元氣箱輕鬆嚐鮮組】台灣好農紅龍果及文旦
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=101131
8513
【私房食譜】茶碗蒸
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=101123
8514
【美濃好米美濃香鑽---高雄147】家常便當
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=101120
8515
【台灣好農元氣箱輕鬆嚐鮮組】菜圃肉蛋包飯
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=101109
8516
【私房食譜】補品之豬腰子
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=101106
8517
【私房食譜】桂圓黑糯紫米糕
名稱重新load
https://www.fooding.com.tw/re

【【好聚。沖繩】水雲&沖繩垣乃花粉末黑糖】【好聚。沖繩】砂鍋水雲鹹粥
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=101091
8597
【美濃好米美濃香鑽---高雄147】簡單絲瓜雞蛋粥
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=114324
8598
【私房食譜】青醬蛤蜊燉飯
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=113996
8599
【私房食譜】雞肉野蕈燉飯Risotto
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=113809
8600
【私房食譜】【餐桌上的年菜】煙燻櫻桃鴨胸佐XO醬香菇燉飯
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=113773
8601
【私房食譜】【餐桌上的年菜】南瓜鴨腿燉飯
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=113576
8602
【私房食譜】焗烤枸杞野蔬燉飯木瓜盅
名稱重新load
https://www.fooding.com.tw/recipe-shares.php?cookid=113576
8603
【私房食譜】焗烤枸杞野蔬燉飯木瓜盅
名稱重新load
http://food.ltn.com.tw/article/8073
8604
杏鮑菇炒肉絲
名稱重新load
http://food.ltn.com.tw/article/8097
8605
節氣食譜 | 蒜香小捲 (影音)
名稱重新load
http://food.ltn.com.tw/article/7873
8606
鳳梨蝦仁蛋炒飯 (影音)
名稱重新load
http://food.ltn.com.tw/article/8004
8607
豬肉番茄炒蛋
名稱重新load
http://food.ltn.com.tw/article/8014
8608
大根燒肉片
名稱重新load
